In [1]:
import os,sys
from src.Mask.Utils import SaveModel,read_yaml,create_directory
from src.Mask.exceptions import CustomException
from src.Mask.loggers import logger
from src.Mask.Constants import *
from dataclasses import dataclass
from pathlib import Path
import tensorflow as tf

In [2]:
#step3) update the entity file: entity file is nothing whatever parameter we have used in yaml file 
#we define them as classvariable 
@dataclass
class TrainedModelConfig():
    root_dirpath:Path
    model_trained_dirpath:Path
    data_dirpath:Path
    all_param:dict
    sequential_model_path:Path


@dataclass
class CallbackConfig():
    #defining the class variable
    root_dirpath:Path
    tensorboard_log_dirpath:Path
    model_checkpoint_path:Path
    all_param:dict

In [3]:
#update the configurationmanager file in this file we read the yaml file and create directories and
#assigning value to the class variable and taking rtn as function
class ConfigurationManager():
    def __init__(self,config_filepath=CONFIG_FILEPATH,param_filepath=PARAM_FILEPATH):
        self.config = read_yaml(config_filepath) #rtn value as configbox dictatonary
        self.param = read_yaml(param_filepath)   #rtn value as configbox dictatonary

        #creating main artifact directory
        create_directory([self.config.artifact_root_dir]) #create artifacts folder 

    def get_call_model_config(self) ->CallbackConfig:
        #initializing the local variable
        config = self.config.prepare_callbacks
        param = self.param
        #getting dirname from this artifacts/callbacks/model_checkpoint/model.h5
        ckpt_directory = os.path.dirname(config.model_checkpoint_path) #create this path artifacts/callbacks/model_checkpoint

        #creating directory of callbacks
        create_directory([config.root_dirpath,ckpt_directory]) #create directoy artifacts/callbacks/model_checkpoint

        #creating an object of class variable and taking rtn as function
        callback_config = CallbackConfig(
            root_dirpath=config.root_dirpath,
            tensorboard_log_dirpath=config.tensorboard_log_dirpath,
            model_checkpoint_path=config.model_checkpoint_path,
            all_param=param
        )
        return callback_config

    def get_trained_model_config(self) ->TrainedModelConfig:
        #initializing the local variable
        config = self.config.model_training
        param = self.param

        #creating the directory model_training
        create_directory([config.root_dirpath]) #it will create model_training directory in project structure

        #creating an object of TrainedModelConfig class variable and assigning value to it and taking rtn as functiom
        trained_model_config = TrainedModelConfig(
            root_dirpath=config.root_dirpath,
            model_trained_dirpath = config.model_trained_dirpath,
            data_dirpath=config.data_dirpath,
            all_param=param,
            sequential_model_path=self.config.prepare_base_model.updated_base_model_path


        )
        return trained_model_config

In [4]:
#step5)updating the component file in this file we create object for class variable and perform task accordingly
from datetime import datetime
class CallbackModel():
    def __init__(self,callbackconfig:CallbackConfig):
        self.callbackconfig = callbackconfig

    @property
    def _get_tensorboard_dir(self):
        timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

        tensorboard_log_dir = os.path.join(self.callbackconfig.tensorboard_log_dirpath,f"TB_{timestamp}.log")

        # Ensure the directory exists
        os.makedirs(os.path.dirname(tensorboard_log_dir), exist_ok=True)

        return tf.keras.callbacks.TensorBoard(
            log_dir=tensorboard_log_dir,
              histogram_freq=1, write_graph=True, write_images=True
        )

    @property
    def _model_checkpoint(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.callbackconfig.model_checkpoint_path,
            monitor="val_loss",
            verbose = 1,
            save_best_only=True
        )

    @property
    def _early_stopping(self):
        return tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            min_delta=self.callbackconfig.all_param.learning_rate,
            patience=2,
            verbose=1,
            mode="auto",
            restore_best_weights=True,
            start_from_epoch=0

        )
    
    #creating main method callback
    def callback(self):
        return [
            self._get_tensorboard_dir,
            self._model_checkpoint,
            self._early_stopping
        ]

In [5]:
#Now updating the component file:- In this file we creating an object of class variable as instance
#and performing task accordingly!!
class ModelTraining():
    def __init__(self,modelconfig:TrainedModelConfig):
        self.modelconfig=modelconfig

    #step1)loading the sequential model at which we have to have to perform training in it
    def load_seq_model(self):
        self.sequential_model = tf.keras.models.load_model(
            filepath=self.modelconfig.sequential_model_path
        )

    #step2) Training the model by using Imagegenerator class or flowing the image through directory of tensorflow
    #Imagegenerator class we used to perform scaling and zooming and also indirect we do preprocessing
    def train_valid_generator(self):
        datagenerator_kwarg = dict(
            rescale=1./255,
            validation_split=0.20
        )

        #creating keyward argument of dataflow_kwarg
        dataflow_kwarg = dict(
            target_size=self.modelconfig.all_param.input_shape[:-1],
            batch_size=self.modelconfig.all_param.batch_size,
            interpolation = "bilinear" #Bilinear interpolation is one of the options for smooth and resizing image.
        )

        #now generating test image by using tensorflow
        valid_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwarg
        )

        #now flowing the valid_image_generator through directory
        self.valid_image_generator = valid_image_generator.flow_from_directory(
            
            directory=self.modelconfig.data_dirpath, #directory where we get image file
            subset='validation',
            shuffle=True,
            **dataflow_kwarg
        )


        #now generating training image through augmentation
        if self.modelconfig.all_param.augmentation:
            train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                **datagenerator_kwarg,
                horizontal_flip=True,
                vertical_flip=True,
                rotation_range=40,
                width_shift_range=0.20,
                height_shift_range=0.20,
                brightness_range=(0.9, 1.1),
                shear_range=0.20,
                zoom_range=0.20,
            )

        else:
            train_image_generator = valid_image_generator

        self.train_image_generator = train_image_generator.flow_from_directory(
            
            directory=self.modelconfig.data_dirpath,
            subset='training',
            shuffle=True,
            **dataflow_kwarg
        )

    #Now training or fitting the model
    def train(self,callback_list:list):
        self.steps_per_epoch = self.train_image_generator.samples // self.train_image_generator.batch_size
        self.validation_steps = self.valid_image_generator.samples // self.valid_image_generator.batch_size

        self.sequential_model.fit(
            self.train_image_generator,
            batch_size = self.modelconfig.all_param.batch_size,
            epochs = self.modelconfig.all_param.epochs,
            verbose = True,
            validation_data = self.valid_image_generator,
            initial_epoch = 0,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            callbacks=callback_list

        )

        #saving the trained model in model checkpoint directory
        SaveModel(filepath=Path(self.modelconfig.model_trained_dirpath),model=self.sequential_model)


In [6]:
%pwd

'd:\\MaskImageDetection\\MaskDetectionModel\\research'

In [7]:
os.chdir('../')
%pwd

'd:\\MaskImageDetection\\MaskDetectionModel'

In [8]:
#step7) updating the training pipeline
try:
    #creating an object of configuration manager class
    cm = ConfigurationManager()
    callback_config = cm.get_call_model_config()

    get_train_config = cm.get_trained_model_config()

    #creating an object of CallbackModel
    cbm = CallbackModel(callback_config)

    lst_obj = cbm.callback()

    #creating an object of training model class
    tm = ModelTraining(get_train_config)

    tm.load_seq_model() #will load sequential model

    tm.train_valid_generator() #this method will generate image flowing through directory

    tm.train(callback_list=lst_obj)

except Exception as e:
    raise CustomException(e,sys)

[2024-10-02 19:37:19,249]-INFO-33-Yaml file config\config.yaml reading
[2024-10-02 19:37:19,258]-INFO-33-Yaml file param.yaml reading
[2024-10-02 19:37:19,264]-INFO-49-Directory artifacts/callbacks created
[2024-10-02 19:37:19,281]-INFO-49-Directory artifacts/callbacks/model_checkpoint created
[2024-10-02 19:37:19,283]-INFO-49-Directory artifacts/model_training created
Found 12 images belonging to 2 classes.
Found 53 images belonging to 2 classes.
Epoch 1/10
5/5 [==============================] - ETA: 0s - loss: 0.7104 - accuracy: 0.5581
Epoch 1: val_loss improved from inf to 0.80436, saving model to artifacts/callbacks/model_checkpoint\model.h5
5/5 [==============================] - 19s 4s/step - loss: 0.7104 - accuracy: 0.5581 - val_loss: 0.8044 - val_accuracy: 0.5000
Epoch 2/10


d:\MaskImageDetection\MaskDetectionModel\Mask\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5/5 [==============================] - ETA: 0s - loss: 0.7622 - accuracy: 0.4651
Epoch 2: val_loss improved from 0.80436 to 0.77147, saving model to artifacts/callbacks/model_checkpoint\model.h5
5/5 [==============================] - 22s 5s/step - loss: 0.7622 - accuracy: 0.4651 - val_loss: 0.7715 - val_accuracy: 0.5000
Epoch 3/10
5/5 [==============================] - ETA: 0s - loss: 0.7273 - accuracy: 0.5581
Epoch 3: val_loss did not improve from 0.77147
5/5 [==============================] - 22s 5s/step - loss: 0.7273 - accuracy: 0.5581 - val_loss: 0.8037 - val_accuracy: 0.4000
Epoch 4/10
5/5 [==============================] - ETA: 0s - loss: 0.6921 - accuracy: 0.4884
Epoch 4: val_loss did not improve from 0.77147
Restoring model weights from the end of the best epoch: 2.
5/5 [==============================] - 22s 4s/step - loss: 0.6921 - accuracy: 0.4884 - val_loss: 0.8045 - val_accuracy: 0.4000
Epoch 4: early stopping
[2024-10-02 19:38:45,976]-INFO-58-Model saved at artifacts\mode